## MODELING

Queremos predecir el precio de venta de la vivienda en base al dataset de venta.

En primer lugar generamos un DataFrame para almacenar los resultados obtenidos de todos los modelos que generemos:

In [640]:
models_results_df = pd.DataFrame(columns=['regressor', 'train_r2_score', 'test_r2_score'])

- Almacenaremos los resultados para los distintos modelos en model_results_df. 



### Bagging & Random Forest Models

Haremos 3 aproximaciones: 

- Modelo 1:  BaggingRegressor y RandomForestRegressor con todas las variables del dataset sin seleccionar variables importantes, sin tratar outliers y sin escalar las variables numéricas.

- Modelo 2: BaggingRegressor y RandomForestRegressor con todas las variables del dataset sin seleccionar variables importantes, tratando outliers (método IQR) y escalando las variables numéricas (MinMaxScaler).

- Modelo 3: BaggingRegressor y RandomForestRegressor seleccionando las variables más importantes (mediante técnicas como PCA, Análisis de Correlación, Información Mutua, ANOVA y ExtraTreesRegressor), tratando outliers (método IQR) y escalando las variables numéricas (MinMaxScaler).

    - Modelo 3.1: BaggingRegressor y RandomForestRegressor seleccionando las variables más importantes (mediante técnicas como PCA, Análisis de Correlación, Información Mutua, ANOVA y ExtraTreesRegressor), tratando outliers (método IQR) pero NO escalando las variables numéricas.

#### Models Pipeline

Creamos la Pipeline con los modelos de Bagging y RF:

In [614]:
class PipelineRunner:
    """A class that runs multiple regression models on given train and test dataframes"""

    def __init__(self, cv_folds: int = CROSS_VALIDATION_FOLDS):
        """
        Args:
            cv_folds: Number of cross validation folds to be used in grid search
        """
        self.cv_folds = cv_folds
        
    def generate_train_test_sets(self, 
                                 df_train: pd.DataFrame, 
                                 df_test: pd.DataFrame, 
                                 target: str) -> Tuple[pd.DataFrame, pd.Series, pd.DataFrame, pd.Series]:
        """
            Generate training and test sets.

            Args:
                df_train (pd.DataFrame): Training dataset.
                df_test (pd.DataFrame): Test dataset.
                target (str): Target variable name.

            Returns:
                X_train (pd.DataFrame): Training features.
                y_train (pd.Series): Training target variable.
                X_test (pd.DataFrame): Test features.
                y_test (pd.Series): Test target variable.
        """
        X_train = df_train.drop(columns=[target])
        y_train = df_train[target]

        X_test = df_test.drop(columns=[target])
        y_test = df_test[target]

        return X_train, y_train, X_test, y_test
        
    def run_pipeline(self, 
                     df_train: pd.DataFrame, 
                     df_test: pd.DataFrame, 
                     target: str) -> GridSearchCV:
        """
        Run multiple regression models on given train and test dataframes.

        Args:
            df_train (pd.DataFrame): Training dataset.
            df_test (pd.DataFrame): Test dataset.
            target (str): Target variable name.

        Returns:
            A GridSearchCV object containing the fitted regression model with the best hyperparameters.
        """
        X_train, y_train, X_test, y_test = self.generate_train_test_sets(df_train, df_test, target)

        models = [
            {
                'name': 'BaggingRegressor',
                'model': BaggingRegressor(DecisionTreeRegressor()),
                'params': {
                    'model__n_estimators': [10, 20, 50, 100, 200]
                }
            },
            {
                'name': 'RandomForestRegressor',
                'model': RandomForestRegressor(),
                'params': {
                    'model__n_estimators': [10, 20, 50, 100, 200],
                    'model__max_depth': [2, 3, 4, 5, 6, 7, 9],
                    'model__min_samples_split': [2, 5, 10]
                }
            }
        ]

        results = []
        for model_info in models:
            model = model_info['model']
            params = model_info['params']

            pipeline = Pipeline(steps=[
                ('model', model)
            ])

            grid_search = GridSearchCV(pipeline, params, cv=self.cv_folds, scoring='r2')
            grid_search.fit(X_train, y_train)

            y_train_pred = grid_search.predict(X_train)
            train_r2_score = r2_score(y_train, y_train_pred)

            y_test_pred = grid_search.predict(X_test)
            test_r2_score = r2_score(y_test, y_test_pred)
            
            results.append({
                'name': model_info['name'],
                'best_params': grid_search.best_params_,
                'best_score': grid_search.best_score_,
                'train_r2_score': train_r2_score,
                'test_r2_score': test_r2_score
            })
            print(f"Model: {model_info['name']}")
            print(f"Best model parameters: {grid_search.best_params_}")
            
            # median R^2 score for the training set across all folds in the cross-validation process.
            print(f"Best cross-validation R^2 score: {grid_search.best_score_:.4f}") 
            
            print(f"Train R^2 score: {train_r2_score:.4f}") 
            print(f"Test R^2 score: {test_r2_score:.4f}")
            print()
    
        best_model = max(results, key=lambda x: x['best_score'])
        print(f"Best model: {best_model['name']}")
        print(f"Best model parameters: {best_model['best_params']}")
        # median R^2 score for the training set across all folds in the cross-validation process.
        print(f"Best cross-validation R^2 score: {best_model['best_score']:.4f}")
        print(f"Train R^2 score: {best_model['train_r2_score']:.4f}") 
        print(f"Test R^2 score: {best_model['test_r2_score']:.4f}")

        return results #grid_search

    def predict(self, df: pd.DataFrame, model: GridSearchCV) -> np.ndarray:
        """
        Make predictions using a fitted regression model.

        Args:
            df (pd.DataFrame): Preprocessed dataset to be used for prediction.
            model (GridSearchCV): Fitted regression model.

        Returns:
            Predictions of the target variable.
        """
        X = df.drop(columns=[model.best_params_['model']])
        return model.predict(X)

#### Modelo 1

Modelo sin escalar los atributos ni hacer una selección de las variables predictoras más relevantes.

- Train dataset: bagging_df1
- Test dataset: bagging_df1_test


##### Train dataset (bagging_df1) y Test dataset (bagging_df1_test)

Debemos codificar las variables categóricas para realizar los modelos de bagging (modelos de bagging solo aceptan variables numéricas).

Para ello utilizaremos:
- one-hot encoding: para las variables que no siguen un orden o jerarquía
- ordinal encoder: para las que hay un order o jerarquía, como floor_cat, property_age_cat y neighborhood_rent_index_cluster. Esto garantiza que los valores codificados reflejen el orden inherente de las categorías y puedan utilizarse para el análisis.

In [615]:
# creamos los datasets para el primer modelo de Bagging Y RF
bagging_df1 = sales_simplified_df.copy(deep=True)
bagging_df1_test = test_data_preprocessed.copy(deep=True)

**Ordinal Encoding**

Codificamos las variables floor_cat, property_age_cat y neighborhood_rent_index_cluster utilizando ordinal encoding. 

Codificaremos la categoría "Unknown" como una etiqueta separada: la sustituimis por -1. Esta es una práctica común cuando se trata de datos desconocidos o faltantes.

In [788]:
bagging_df1.floor_cat.value_counts()

0 - 4      2359
Unknown    1680
5 - 11      623
+11          15
Name: floor_cat, dtype: int64

In [789]:
bagging_df1.property_age_cat.value_counts()

40 - 70      1933
Unknown      1415
70 - 120      500
0 - 40        389
120 - 150     300
+150          140
Name: property_age_cat, dtype: int64

In [790]:
bagging_df1.neighborhood_rent_index_cluster.value_counts()

0 - 2337       1571
3371 - 5042    1403
2337 - 3371    1060
+5042           643
Name: neighborhood_rent_index_cluster, dtype: int64

In [616]:
# variables que queremos codificar usando ordinal encoder
oe_cols = oe_cols = ['floor_cat', # unknown
                     'property_age_cat', # unknown
                     'neighborhood_rent_index_cluster' 
                    ]
# Define the order of the categories for each column
floor_cat_order = ['Unknown', '0 - 4', '5 - 11', '+11']
property_age_cat_order = ['Unknown', '0 - 40', '40 - 70', '70 - 120', '120 - 150', '+150']
neighborhood_rent_index_cluster_order = ['Unknown', '0 - 2337', '2337 - 3371', '3371 - 5042', '+5042']

# Create an instance of the ordinal encoder
encoder = OrdinalEncoder(categories=[floor_cat_order, 
                                     property_age_cat_order, 
                                     neighborhood_rent_index_cluster_order])

for col in oe_cols:
    # Create a new column name for the encoded data
    new_col = col + '_encoded'
    # Create an instance of the ordinal encoder
    encoder = OrdinalEncoder(categories=[eval(col+'_order')])
    # Fit and transform the selected columns
    bagging_df1[new_col] = encoder.fit_transform(bagging_df1[[col]])
    bagging_df1_test[new_col] = encoder.fit_transform(bagging_df1_test[[col]])
    # replace 0 by -1 ('Unknown' by -1)
    bagging_df1[new_col] = bagging_df1[new_col].replace(0, -1)
    bagging_df1_test[new_col] = bagging_df1_test[new_col].replace(0, -1)


# Print the encoded data
display(bagging_df1[['floor_cat','floor_cat_encoded','property_age_cat','property_age_cat_encoded','neighborhood_rent_index_cluster','neighborhood_rent_index_cluster_encoded']].head())
display(bagging_df1_test[['floor_cat','floor_cat_encoded','property_age_cat','property_age_cat_encoded','neighborhood_rent_index_cluster','neighborhood_rent_index_cluster_encoded']].head())

,floor_cat,floor_cat_encoded,property_age_cat,property_age_cat_encoded,neighborhood_rent_index_cluster,neighborhood_rent_index_cluster_encoded
0,0 - 4,1.0,0 - 40,1.0,3371 - 5042,3.0
1,Unknown,-1.0,40 - 70,2.0,0 - 2337,1.0
2,0 - 4,1.0,Unknown,-1.0,0 - 2337,1.0
3,Unknown,-1.0,40 - 70,2.0,3371 - 5042,3.0
4,5 - 11,2.0,70 - 120,3.0,3371 - 5042,3.0


,floor_cat,floor_cat_encoded,property_age_cat,property_age_cat_encoded,neighborhood_rent_index_cluster,neighborhood_rent_index_cluster_encoded
0,0 - 4,1.0,40 - 70,2.0,2337 - 3371,2.0
1,Unknown,-1.0,70 - 120,3.0,3371 - 5042,3.0
2,5 - 11,2.0,40 - 70,2.0,3371 - 5042,3.0
3,0 - 4,1.0,40 - 70,2.0,2337 - 3371,2.0
4,0 - 4,1.0,70 - 120,3.0,+5042,4.0


- Ya tenemos las variables codificadas usando ordinal encoding. 

**One-hot Encoding**

Codificamos las variables binarias junto con orientation, heating y property_type utilizando OHE.

In [617]:
def one_hot_encode(df: pd.DataFrame, cat_features: List[str]) -> pd.DataFrame:
    """
    Encodes categorical features in a dataframe using scikit-learn's OneHotEncoder.

    Args:
    df (pandas.DataFrame): input dataframe
    cat_features (list): list of categorical feature column names

    Returns:
    pandas.DataFrame: dataframe with categorical features one-hot encoded
    """
    # create OneHotEncoder object and fit on categorical features
    encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
    encoder.fit(df[cat_features])

    # get category names from encoder
    cat_names = encoder.get_feature_names_out(cat_features)

    # transform categorical features to one-hot encoded features and create new dataframe
    encoded_cat_features = encoder.transform(df[cat_features])
    df_encoded = pd.DataFrame(encoded_cat_features, columns=cat_names)

    # add original numerical features to encoded features dataframe
    num_features = df.drop(cat_features, axis=1)
    df_encoded = pd.concat([num_features.reset_index(drop=True), df_encoded], axis=1)

    return df_encoded


In [618]:
# variables que queremos codificar usando OHE
ohe_cols = get_binary_cols(sales_simplified_df) + ['orientation', 'heating', 'property_type']
# binary variables:
    # ['balcony',
    #  'terrace',
    #  'exterior',
    #  'rooftop',
    #  'elevator',
    #  'pool',
    #  'ac',
    #  'garage',
    #  'garden']

In [619]:
# aplicamos one - hot encoding al dataset
bagging_df1 = one_hot_encode(bagging_df1, ohe_cols)
bagging_df1_test = one_hot_encode(bagging_df1_test, ohe_cols)


display(bagging_df1.head())
display(bagging_df1.shape)

display(bagging_df1_test.head())
display(bagging_df1_test.shape)

`sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
`sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.


,price,latitude,longitude,sq_meters_built,rooms,bathrooms,neighborhood,dist_city_center,dist_closest_station,property_age_cat,floor_cat,neighborhood_rent_index_cluster,rooms_per_100_sqm,bathrooms_per_100_sqm,floor_cat_encoded,property_age_cat_encoded,neighborhood_rent_index_cluster_encoded,balcony_0,balcony_1,terrace_0,terrace_1,exterior_0,exterior_1,rooftop_0,rooftop_1,elevator_0,elevator_1,pool_0,pool_1,ac_0,ac_1,garage_0,garage_1,garden_0,garden_1,orientation_ este,orientation_ norte,orientation_ oeste,orientation_ sur,orientation_Unknown,heating_Unknown,heating_bomba,heating_electric,heating_gas,heating_individual,property_type_atico,property_type_chalet,property_type_chalet_adosado,property_type_chalet_independiente,property_type_chalet_pareado,property_type_duplex,property_type_estudio,property_type_piso
0,2290000,41.409071,2.099850,532,5,6,Vallvidrera - El Tibidabo i les Planes,6.302847,2.949904,0 - 40,0 - 4,3371 - 5042,0.939850,1.127820,1.0,1.0,3.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,90000,41.434545,2.171110,50,1,1,Can Peguera - El Turó de la Peira,5.189265,0.329115,40 - 70,Unknown,0 - 2337,2.000000,2.000000,-1.0,2.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,145000,41.444518,2.175309,53,3,1,Verdun,6.313668,0.284071,Unknown,0 - 4,0 - 2337,5.660377,1.886792,1.0,-1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,675000,41.392209,2.153368,120,4,2,L'Antiga Esquerra de l'Eixample,1.462060,0.443717,40 - 70,Unknown,3371 - 5042,3.333333,1.666667,-1.0,2.0,3.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,319000,41.413385,2.162246,69,3,1,El Baix Guinardó,2.906147,0.371854,70 - 120,5 - 11,3371 - 5042,4.347826,1.449275,2.0,3.0,3.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


(4675, 53)

,price,latitude,longitude,sq_meters_built,rooms,bathrooms,neighborhood,dist_city_center,dist_closest_station,property_age_cat,floor_cat,neighborhood_rent_index_cluster,rooms_per_100_sqm,bathrooms_per_100_sqm,floor_cat_encoded,property_age_cat_encoded,neighborhood_rent_index_cluster_encoded,balcony_0,balcony_1,terrace_0,terrace_1,exterior_0,exterior_1,rooftop_0,rooftop_1,elevator_0,elevator_1,pool_0,pool_1,ac_0,ac_1,garage_0,garage_1,garden_0,garden_1,orientation_ este,orientation_ norte,orientation_ oeste,orientation_ sur,orientation_Unknown,heating_Unknown,heating_bomba,heating_electric,heating_gas,heating_individual,property_type_atico,property_type_chalet,property_type_chalet_adosado,property_type_chalet_independiente,property_type_chalet_pareado,property_type_duplex,property_type_estudio,property_type_piso
0,133000,41.421077,2.213120,76,3,1,Sant Martí,5.158288,0.272653,40 - 70,0 - 4,2337 - 3371,3.947368,1.315789,1.0,2.0,2.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,485000,41.380788,2.152472,130,4,2,La Nova Esquerra de l'Eixample,1.657143,0.245719,70 - 120,Unknown,3371 - 5042,3.076923,1.538462,-1.0,3.0,3.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,265000,41.406803,2.183533,77,3,1,El Camp de l'Arpa del Clot,2.389801,0.084520,40 - 70,5 - 11,3371 - 5042,3.896104,1.298701,2.0,2.0,3.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,154500,41.433890,2.172277,61,2,1,Porta,5.119194,0.355555,40 - 70,0 - 4,2337 - 3371,3.278689,1.639344,1.0,2.0,2.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,1200000,41.393411,2.145017,120,3,2,Sant Gervasi - Galvany,2.166588,0.735841,70 - 120,0 - 4,+5042,2.500000,1.666667,1.0,3.0,4.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


(1170, 53)

Eliminamos las variables categóricas no codificadas: 

In [620]:
no_encoded_cat_vars = bagging_df1.select_dtypes(include=['category']).columns.tolist()
no_encoded_cat_vars

['neighborhood',
 'property_age_cat',
 'floor_cat',
 'neighborhood_rent_index_cluster']

In [621]:
no_encoded_cat_vars_test = bagging_df1_test.select_dtypes(include=['category']).columns.tolist()
no_encoded_cat_vars_test

['neighborhood',
 'property_age_cat',
 'floor_cat',
 'neighborhood_rent_index_cluster']

In [622]:
bagging_df1.drop(no_encoded_cat_vars, axis=1, inplace=True)
bagging_df1_test.drop(no_encoded_cat_vars_test, axis=1, inplace=True)

In [623]:
# exploración rápida del df
print("TRAIN: ")
explore_data(bagging_df1)
print("\n\n")
print("TEST: ")
explore_data(bagging_df1_test)

TRAIN: 
The dataset includes 4675 instances (rows) and 49 variables (columns).



,price,latitude,longitude,sq_meters_built,rooms,bathrooms,dist_city_center,dist_closest_station,rooms_per_100_sqm,bathrooms_per_100_sqm,floor_cat_encoded,property_age_cat_encoded,neighborhood_rent_index_cluster_encoded,balcony_0,balcony_1,terrace_0,terrace_1,exterior_0,exterior_1,rooftop_0,rooftop_1,elevator_0,elevator_1,pool_0,pool_1,ac_0,ac_1,garage_0,garage_1,garden_0,garden_1,orientation_ este,orientation_ norte,orientation_ oeste,orientation_ sur,orientation_Unknown,heating_Unknown,heating_bomba,heating_electric,heating_gas,heating_individual,property_type_atico,property_type_chalet,property_type_chalet_adosado,property_type_chalet_independiente,property_type_chalet_pareado,property_type_duplex,property_type_estudio,property_type_piso
0,2290000,41.409071,2.099850,532,5,6,6.302847,2.949904,0.939850,1.127820,1.0,1.0,3.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,90000,41.434545,2.171110,50,1,1,5.189265,0.329115,2.000000,2.000000,-1.0,2.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,145000,41.444518,2.175309,53,3,1,6.313668,0.284071,5.660377,1.886792,1.0,-1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,675000,41.392209,2.153368,120,4,2,1.462060,0.443717,3.333333,1.666667,-1.0,2.0,3.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,319000,41.413385,2.162246,69,3,1,2.906147,0.371854,4.347826,1.449275,2.0,3.0,3.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0




<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4675 entries, 0 to 4674
Data columns (total 49 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   price                                    4675 non-null   int64  
 1   latitude                                 4675 non-null   float64
 2   longitude                                4675 non-null   float64
 3   sq_meters_built                          4675 non-null   int64  
 4   rooms                                    4675 non-null   int64  
 5   bathrooms                                4675 non-null   int64  
 6   dist_city_center                         4675 non-null   float64
 7   dist_closest_station                     4675 non-null   float64
 8   rooms_per_100_sqm                        4675 non-null   float64
 9   bathrooms_per_100_sqm                    4675 non-null   float64
 10  floor_cat_encoded                        4675 

,price,latitude,longitude,sq_meters_built,rooms,bathrooms,dist_city_center,dist_closest_station,rooms_per_100_sqm,bathrooms_per_100_sqm,floor_cat_encoded,property_age_cat_encoded,neighborhood_rent_index_cluster_encoded,balcony_0,balcony_1,terrace_0,terrace_1,exterior_0,exterior_1,rooftop_0,rooftop_1,elevator_0,elevator_1,pool_0,pool_1,ac_0,ac_1,garage_0,garage_1,garden_0,garden_1,orientation_ este,orientation_ norte,orientation_ oeste,orientation_ sur,orientation_Unknown,heating_Unknown,heating_bomba,heating_electric,heating_gas,heating_individual,property_type_atico,property_type_chalet,property_type_chalet_adosado,property_type_chalet_independiente,property_type_chalet_pareado,property_type_duplex,property_type_estudio,property_type_piso
count,4.675000e+03,4675.000000,4675.000000,4675.000000,4675.000000,4675.000000,4675.000000,4675.000000,4675.000000,4675.000000,4675.000000,4675.000000,4675.000000,4675.000000,4675.000000,4675.000000,4675.000000,4675.000000,4675.000000,4675.000000,4675.000000,4675.000000,4675.000000,4675.000000,4675.000000,4675.000000,4675.000000,4675.000000,4675.000000,4675.000000,4675.000000,4675.000000,4675.000000,4675.000000,4675.000000,4675.000000,4675.000000,4675.000000,4675.000000,4675.000000,4675.000000,4675.000000,4675.000000,4675.000000,4675.000000,4675.000000,4675.000000,4675.000000,4675.000000
mean,3.567950e+05,41.406603,2.168282,85.819465,2.580749,1.410481,3.501909,0.359388,3.379902,1.844308,0.421390,1.333904,2.238717,0.623957,0.376043,0.760428,0.239572,0.293262,0.706738,0.930695,0.069305,0.436791,0.563209,0.973690,0.026310,0.549091,0.450909,0.931551,0.068449,0.989733,0.010267,0.118289,0.125775,0.050053,0.245775,0.460107,0.589733,0.056898,0.042567,0.235936,0.074866,0.050267,0.007701,0.005989,0.005561,0.002567,0.016043,0.037861,0.874011
std,4.910966e+05,0.026137,0.022753,66.934320,1.160086,0.793276,2.034103,0.280663,1.693352,1.088397,1.117207,1.731172,1.062609,0.484443,0.484443,0.426868,0.426868,0.455306,0.455306,0.253999,0.253999,0.496042,0.496042,0.160073,0.160073,0.497637,0.497637,0.252542,0.252542,0.100817,0.100817,0.322984,0.331632,0.218079,0.430592,0.498459,0.491935,0.231673,0.201900,0.424628,0.263204,0.218520,0.087424,0.077167,0.074376,0.050604,0.125653,0.190880,0.331873
min,2.800000e+04,41.355002,2.092727,13.000000,0.000000,1.000000,0.075001,0.003595,0.000000,0.227273,-1.000000,-1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.350000e+05,41.382507,2.153545,55.000000,2.000000,1.000000,1.748592,0.212796,2.439024,1.428571,-1.000000,-1.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,1.870000e+05,41.404373,2.170370,68.000000,3.000000,1.000000,3.361583,0.311358,3.448276,1.666667,1.000000,2.000000,2.000000,1.000000,0.000000,1.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,3.900000e+05,41.424739,2.180816,92.000000,3.000000,2.000000,4.914956,0.420666,4.347826,2.083333,1.000000,2.000000,3.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


 Number of duplicates: 4



TEST: 
The dataset includes 1170 instances (rows) and 49 variables (columns).



,price,latitude,longitude,sq_meters_built,rooms,bathrooms,dist_city_center,dist_closest_station,rooms_per_100_sqm,bathrooms_per_100_sqm,floor_cat_encoded,property_age_cat_encoded,neighborhood_rent_index_cluster_encoded,balcony_0,balcony_1,terrace_0,terrace_1,exterior_0,exterior_1,rooftop_0,rooftop_1,elevator_0,elevator_1,pool_0,pool_1,ac_0,ac_1,garage_0,garage_1,garden_0,garden_1,orientation_ este,orientation_ norte,orientation_ oeste,orientation_ sur,orientation_Unknown,heating_Unknown,heating_bomba,heating_electric,heating_gas,heating_individual,property_type_atico,property_type_chalet,property_type_chalet_adosado,property_type_chalet_independiente,property_type_chalet_pareado,property_type_duplex,property_type_estudio,property_type_piso
0,133000,41.421077,2.213120,76,3,1,5.158288,0.272653,3.947368,1.315789,1.0,2.0,2.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,485000,41.380788,2.152472,130,4,2,1.657143,0.245719,3.076923,1.538462,-1.0,3.0,3.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,265000,41.406803,2.183533,77,3,1,2.389801,0.084520,3.896104,1.298701,2.0,2.0,3.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,154500,41.433890,2.172277,61,2,1,5.119194,0.355555,3.278689,1.639344,1.0,2.0,2.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,1200000,41.393411,2.145017,120,3,2,2.166588,0.735841,2.500000,1.666667,1.0,3.0,4.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0




<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1170 entries, 0 to 1169
Data columns (total 49 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   price                                    1170 non-null   int64  
 1   latitude                                 1170 non-null   float64
 2   longitude                                1170 non-null   float64
 3   sq_meters_built                          1170 non-null   int64  
 4   rooms                                    1170 non-null   int64  
 5   bathrooms                                1170 non-null   int64  
 6   dist_city_center                         1170 non-null   float64
 7   dist_closest_station                     1170 non-null   float64
 8   rooms_per_100_sqm                        1170 non-null   float64
 9   bathrooms_per_100_sqm                    1170 non-null   float64
 10  floor_cat_encoded                        1170 

,price,latitude,longitude,sq_meters_built,rooms,bathrooms,dist_city_center,dist_closest_station,rooms_per_100_sqm,bathrooms_per_100_sqm,floor_cat_encoded,property_age_cat_encoded,neighborhood_rent_index_cluster_encoded,balcony_0,balcony_1,terrace_0,terrace_1,exterior_0,exterior_1,rooftop_0,rooftop_1,elevator_0,elevator_1,pool_0,pool_1,ac_0,ac_1,garage_0,garage_1,garden_0,garden_1,orientation_ este,orientation_ norte,orientation_ oeste,orientation_ sur,orientation_Unknown,heating_Unknown,heating_bomba,heating_electric,heating_gas,heating_individual,property_type_atico,property_type_chalet,property_type_chalet_adosado,property_type_chalet_independiente,property_type_chalet_pareado,property_type_duplex,property_type_estudio,property_type_piso
count,1.170000e+03,1170.000000,1170.000000,1170.000000,1170.000000,1170.000000,1170.000000,1170.000000,1170.000000,1170.000000,1170.000000,1170.000000,1170.000000,1170.000000,1170.000000,1170.000000,1170.000000,1170.000000,1170.000000,1170.000000,1170.000000,1170.000000,1170.000000,1170.000000,1170.000000,1170.000000,1170.000000,1170.000000,1170.000000,1170.000000,1170.000000,1170.000000,1170.000000,1170.000000,1170.000000,1170.000000,1170.000000,1170.000000,1170.000000,1170.000000,1170.000000,1170.000000,1170.000000,1170.000000,1170.000000,1170.000000,1170.000000,1170.000000,1170.000000
mean,4.050583e+05,41.406164,2.167129,89.911966,2.647863,1.453846,3.494901,0.367251,3.403759,1.847461,0.452137,1.335897,2.361538,0.614530,0.385470,0.737607,0.262393,0.264103,0.735897,0.929060,0.070940,0.400000,0.600000,0.960684,0.039316,0.537607,0.462393,0.929060,0.070940,0.990598,0.009402,0.100000,0.136752,0.050427,0.234188,0.478632,0.610256,0.052137,0.048718,0.210256,0.078632,0.053846,0.004274,0.004274,0.007692,0.004274,0.011966,0.037607,0.876068
std,6.442154e+05,0.025809,0.023118,67.150712,1.130085,0.844652,1.980491,0.295759,2.202354,1.893766,1.109805,1.722640,1.077839,0.486914,0.486914,0.440123,0.440123,0.441043,0.441043,0.256835,0.256835,0.490107,0.490107,0.194429,0.194429,0.498797,0.498797,0.256835,0.256835,0.096547,0.096547,0.300128,0.343732,0.218919,0.423671,0.499757,0.487901,0.222398,0.215370,0.407665,0.269279,0.225810,0.065260,0.065260,0.087405,0.065260,0.108778,0.190325,0.329644
min,2.900000e+04,41.355143,2.099004,10.000000,0.000000,1.000000,0.074264,0.014215,0.000000,0.558659,-1.000000,-1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.396000e+05,41.383035,2.152141,57.000000,2.000000,1.000000,1.782861,0.217491,2.439024,1.388889,-1.000000,-1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,2.025000e+05,41.403535,2.169455,70.000000,3.000000,1.000000,3.415416,0.315010,3.380309,1.666667,1.000000,2.000000,2.000000,1.000000,0.000000,1.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,4.147500e+05,41.424744,2.180327,100.000000,3.000000,2.000000,4.787665,0.429002,4.347826,2.000000,1.000000,2.000000,3.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000


 Number of duplicates: 0


- No tenemos valores nulos y todas las variables son numéricas. 

##### Model 1: Bagging & RF 

In [624]:
# create an instance of PipelineRunner
pipeline_runner = PipelineRunner()
# call the run_pipeline() method, passing in the dataframes and target variable name
results = pipeline_runner.run_pipeline(df_train=bagging_df1, 
                                       df_test=bagging_df1_test, 
                                       target=TARGET_VARIABLE)
# examine the results for each model
results

Model: BaggingRegressor
Best model parameters: {'model__n_estimators': 100}
Best cross-validation R^2 score: 0.8413
Train R^2 score: 0.9767
Test R^2 score: 0.7393

Model: RandomForestRegressor
Best model parameters: {'model__max_depth': 9, 'model__min_samples_split': 10, 'model__n_estimators': 20}
Best cross-validation R^2 score: 0.8376
Train R^2 score: 0.9392
Test R^2 score: 0.7231

Best model: BaggingRegressor
Best model parameters: {'model__n_estimators': 100}
Best cross-validation R^2 score: 0.8413
Train R^2 score: 0.9767
Test R^2 score: 0.7393


[{'name': 'BaggingRegressor',
  'best_params': {'model__n_estimators': 100},
  'best_score': 0.8413459550193905,
  'train_r2_score': 0.9766845778640442,
  'test_r2_score': 0.739290218306422},
 {'name': 'RandomForestRegressor',
  'best_params': {'model__max_depth': 9,
   'model__min_samples_split': 10,
   'model__n_estimators': 20},
  'best_score': 0.8376489876860275,
  'train_r2_score': 0.9392475407127426,
  'test_r2_score': 0.7231028416313489}]

In [655]:
# Loop through each model in results
for model in results:
    model_name = model['name']+'_1'
    train_r2_score = model['train_r2_score']
    test_r2_score = model['test_r2_score']
    
    # Append the results to the DataFrame
    models_results_df = pd.concat([models_results_df, pd.DataFrame({'regressor': model_name, 
                                                                   'train_r2_score': train_r2_score,
                                                                   'test_r2_score': test_r2_score},
                                                                   index=[0])], ignore_index=True)
# Display the DataFrame
display(models_results_df)

,regressor,train_r2_score,test_r2_score
0,BaggingRegressor_1,0.976685,0.739290
1,RandomForestRegressor_1,0.939248,0.723103


#### Modelo 2

En los modelos anteriores (BaggingRegressor y RandomForestRegressor) nos se han escalado las variables, no se han quitado los outliers y no se ha hecho una selección de variables.

Por tanto, ahora escalaremos las variables numéricas y trataremos los outliers. La selección de variables predictoras más importantes la realizaremos en el tercer modelo. 


#####  Train dataset (bagging_df2) y Test dataset (bagging_df2_test)

Primero, realizamos el mismo preprocesamiento que en el primer modelo: 

In [626]:
# creamos los datasets para el primer modelo de Bagging
bagging_df2 = sales_simplified_df.copy(deep=True)
bagging_df2_test = test_data_preprocessed.copy(deep=True)


# variables que queremos codificar usando ordinal encoder
oe_cols = oe_cols = ['floor_cat', # unknown
                     'property_age_cat', # unknown
                     'neighborhood_rent_index_cluster' 
                    ]
# Define the order of the categories for each column
floor_cat_order = ['Unknown', '0 - 4', '5 - 11', '+11']
property_age_cat_order = ['Unknown', '0 - 40', '40 - 70', '70 - 120', '120 - 150', '+150']
neighborhood_rent_index_cluster_order = ['Unknown', '0 - 2337', '2337 - 3371', '3371 - 5042', '+5042']

# Create an instance of the ordinal encoder
encoder = OrdinalEncoder(categories=[floor_cat_order, 
                                     property_age_cat_order, 
                                     neighborhood_rent_index_cluster_order])

for col in oe_cols:
    # Create a new column name for the encoded data
    new_col = col + '_encoded'
    # Create an instance of the ordinal encoder
    encoder = OrdinalEncoder(categories=[eval(col+'_order')])
    # Fit and transform the selected columns
    bagging_df2[new_col] = encoder.fit_transform(bagging_df2[[col]])
    bagging_df2_test[new_col] = encoder.fit_transform(bagging_df2_test[[col]])
    # replace 0 by -1 ('Unknown' by -1)
    bagging_df2[new_col] = bagging_df2[new_col].replace(0, -1)
    bagging_df2_test[new_col] = bagging_df2_test[new_col].replace(0, -1)

# variables que queremos codificar usando OHE
ohe_cols = get_binary_cols(sales_simplified_df) + ['orientation', 'heating', 'property_type']
# binary variables:
    # ['balcony',
    #  'terrace',
    #  'exterior',
    #  'rooftop',
    #  'elevator',
    #  'pool',
    #  'ac',
    #  'garage',
    #  'garden']

# aplicamos one - hot encoding al dataset
bagging_df2 = one_hot_encode(bagging_df2, ohe_cols)
bagging_df2_test = one_hot_encode(bagging_df2_test, ohe_cols)

# eliminamos variables categóricas no codificadas
no_encoded_cat_vars = bagging_df2.select_dtypes(include=['category']).columns.tolist()
bagging_df2.drop(no_encoded_cat_vars, axis=1, inplace=True)

no_encoded_cat_vars_test = bagging_df2_test.select_dtypes(include=['category']).columns.tolist()
bagging_df2_test.drop(no_encoded_cat_vars_test, axis=1, inplace=True)

# exploración rápida del df
print("TRAIN: ")
explore_data(bagging_df2)
print("\n\n")
print("TEST: ")
explore_data(bagging_df2_test)

TRAIN: 
The dataset includes 4675 instances (rows) and 49 variables (columns).



`sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
`sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.


,price,latitude,longitude,sq_meters_built,rooms,bathrooms,dist_city_center,dist_closest_station,rooms_per_100_sqm,bathrooms_per_100_sqm,floor_cat_encoded,property_age_cat_encoded,neighborhood_rent_index_cluster_encoded,balcony_0,balcony_1,terrace_0,terrace_1,exterior_0,exterior_1,rooftop_0,rooftop_1,elevator_0,elevator_1,pool_0,pool_1,ac_0,ac_1,garage_0,garage_1,garden_0,garden_1,orientation_ este,orientation_ norte,orientation_ oeste,orientation_ sur,orientation_Unknown,heating_Unknown,heating_bomba,heating_electric,heating_gas,heating_individual,property_type_atico,property_type_chalet,property_type_chalet_adosado,property_type_chalet_independiente,property_type_chalet_pareado,property_type_duplex,property_type_estudio,property_type_piso
0,2290000,41.409071,2.099850,532,5,6,6.302847,2.949904,0.939850,1.127820,1.0,1.0,3.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,90000,41.434545,2.171110,50,1,1,5.189265,0.329115,2.000000,2.000000,-1.0,2.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,145000,41.444518,2.175309,53,3,1,6.313668,0.284071,5.660377,1.886792,1.0,-1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,675000,41.392209,2.153368,120,4,2,1.462060,0.443717,3.333333,1.666667,-1.0,2.0,3.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,319000,41.413385,2.162246,69,3,1,2.906147,0.371854,4.347826,1.449275,2.0,3.0,3.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0




<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4675 entries, 0 to 4674
Data columns (total 49 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   price                                    4675 non-null   int64  
 1   latitude                                 4675 non-null   float64
 2   longitude                                4675 non-null   float64
 3   sq_meters_built                          4675 non-null   int64  
 4   rooms                                    4675 non-null   int64  
 5   bathrooms                                4675 non-null   int64  
 6   dist_city_center                         4675 non-null   float64
 7   dist_closest_station                     4675 non-null   float64
 8   rooms_per_100_sqm                        4675 non-null   float64
 9   bathrooms_per_100_sqm                    4675 non-null   float64
 10  floor_cat_encoded                        4675 

,price,latitude,longitude,sq_meters_built,rooms,bathrooms,dist_city_center,dist_closest_station,rooms_per_100_sqm,bathrooms_per_100_sqm,floor_cat_encoded,property_age_cat_encoded,neighborhood_rent_index_cluster_encoded,balcony_0,balcony_1,terrace_0,terrace_1,exterior_0,exterior_1,rooftop_0,rooftop_1,elevator_0,elevator_1,pool_0,pool_1,ac_0,ac_1,garage_0,garage_1,garden_0,garden_1,orientation_ este,orientation_ norte,orientation_ oeste,orientation_ sur,orientation_Unknown,heating_Unknown,heating_bomba,heating_electric,heating_gas,heating_individual,property_type_atico,property_type_chalet,property_type_chalet_adosado,property_type_chalet_independiente,property_type_chalet_pareado,property_type_duplex,property_type_estudio,property_type_piso
count,4.675000e+03,4675.000000,4675.000000,4675.000000,4675.000000,4675.000000,4675.000000,4675.000000,4675.000000,4675.000000,4675.000000,4675.000000,4675.000000,4675.000000,4675.000000,4675.000000,4675.000000,4675.000000,4675.000000,4675.000000,4675.000000,4675.000000,4675.000000,4675.000000,4675.000000,4675.000000,4675.000000,4675.000000,4675.000000,4675.000000,4675.000000,4675.000000,4675.000000,4675.000000,4675.000000,4675.000000,4675.000000,4675.000000,4675.000000,4675.000000,4675.000000,4675.000000,4675.000000,4675.000000,4675.000000,4675.000000,4675.000000,4675.000000,4675.000000
mean,3.567950e+05,41.406603,2.168282,85.819465,2.580749,1.410481,3.501909,0.359388,3.379902,1.844308,0.421390,1.333904,2.238717,0.623957,0.376043,0.760428,0.239572,0.293262,0.706738,0.930695,0.069305,0.436791,0.563209,0.973690,0.026310,0.549091,0.450909,0.931551,0.068449,0.989733,0.010267,0.118289,0.125775,0.050053,0.245775,0.460107,0.589733,0.056898,0.042567,0.235936,0.074866,0.050267,0.007701,0.005989,0.005561,0.002567,0.016043,0.037861,0.874011
std,4.910966e+05,0.026137,0.022753,66.934320,1.160086,0.793276,2.034103,0.280663,1.693352,1.088397,1.117207,1.731172,1.062609,0.484443,0.484443,0.426868,0.426868,0.455306,0.455306,0.253999,0.253999,0.496042,0.496042,0.160073,0.160073,0.497637,0.497637,0.252542,0.252542,0.100817,0.100817,0.322984,0.331632,0.218079,0.430592,0.498459,0.491935,0.231673,0.201900,0.424628,0.263204,0.218520,0.087424,0.077167,0.074376,0.050604,0.125653,0.190880,0.331873
min,2.800000e+04,41.355002,2.092727,13.000000,0.000000,1.000000,0.075001,0.003595,0.000000,0.227273,-1.000000,-1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.350000e+05,41.382507,2.153545,55.000000,2.000000,1.000000,1.748592,0.212796,2.439024,1.428571,-1.000000,-1.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,1.870000e+05,41.404373,2.170370,68.000000,3.000000,1.000000,3.361583,0.311358,3.448276,1.666667,1.000000,2.000000,2.000000,1.000000,0.000000,1.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,3.900000e+05,41.424739,2.180816,92.000000,3.000000,2.000000,4.914956,0.420666,4.347826,2.083333,1.000000,2.000000,3.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


 Number of duplicates: 4



TEST: 
The dataset includes 1170 instances (rows) and 49 variables (columns).



,price,latitude,longitude,sq_meters_built,rooms,bathrooms,dist_city_center,dist_closest_station,rooms_per_100_sqm,bathrooms_per_100_sqm,floor_cat_encoded,property_age_cat_encoded,neighborhood_rent_index_cluster_encoded,balcony_0,balcony_1,terrace_0,terrace_1,exterior_0,exterior_1,rooftop_0,rooftop_1,elevator_0,elevator_1,pool_0,pool_1,ac_0,ac_1,garage_0,garage_1,garden_0,garden_1,orientation_ este,orientation_ norte,orientation_ oeste,orientation_ sur,orientation_Unknown,heating_Unknown,heating_bomba,heating_electric,heating_gas,heating_individual,property_type_atico,property_type_chalet,property_type_chalet_adosado,property_type_chalet_independiente,property_type_chalet_pareado,property_type_duplex,property_type_estudio,property_type_piso
0,133000,41.421077,2.213120,76,3,1,5.158288,0.272653,3.947368,1.315789,1.0,2.0,2.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,485000,41.380788,2.152472,130,4,2,1.657143,0.245719,3.076923,1.538462,-1.0,3.0,3.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,265000,41.406803,2.183533,77,3,1,2.389801,0.084520,3.896104,1.298701,2.0,2.0,3.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,154500,41.433890,2.172277,61,2,1,5.119194,0.355555,3.278689,1.639344,1.0,2.0,2.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,1200000,41.393411,2.145017,120,3,2,2.166588,0.735841,2.500000,1.666667,1.0,3.0,4.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0




<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1170 entries, 0 to 1169
Data columns (total 49 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   price                                    1170 non-null   int64  
 1   latitude                                 1170 non-null   float64
 2   longitude                                1170 non-null   float64
 3   sq_meters_built                          1170 non-null   int64  
 4   rooms                                    1170 non-null   int64  
 5   bathrooms                                1170 non-null   int64  
 6   dist_city_center                         1170 non-null   float64
 7   dist_closest_station                     1170 non-null   float64
 8   rooms_per_100_sqm                        1170 non-null   float64
 9   bathrooms_per_100_sqm                    1170 non-null   float64
 10  floor_cat_encoded                        1170 

,price,latitude,longitude,sq_meters_built,rooms,bathrooms,dist_city_center,dist_closest_station,rooms_per_100_sqm,bathrooms_per_100_sqm,floor_cat_encoded,property_age_cat_encoded,neighborhood_rent_index_cluster_encoded,balcony_0,balcony_1,terrace_0,terrace_1,exterior_0,exterior_1,rooftop_0,rooftop_1,elevator_0,elevator_1,pool_0,pool_1,ac_0,ac_1,garage_0,garage_1,garden_0,garden_1,orientation_ este,orientation_ norte,orientation_ oeste,orientation_ sur,orientation_Unknown,heating_Unknown,heating_bomba,heating_electric,heating_gas,heating_individual,property_type_atico,property_type_chalet,property_type_chalet_adosado,property_type_chalet_independiente,property_type_chalet_pareado,property_type_duplex,property_type_estudio,property_type_piso
count,1.170000e+03,1170.000000,1170.000000,1170.000000,1170.000000,1170.000000,1170.000000,1170.000000,1170.000000,1170.000000,1170.000000,1170.000000,1170.000000,1170.000000,1170.000000,1170.000000,1170.000000,1170.000000,1170.000000,1170.000000,1170.000000,1170.000000,1170.000000,1170.000000,1170.000000,1170.000000,1170.000000,1170.000000,1170.000000,1170.000000,1170.000000,1170.000000,1170.000000,1170.000000,1170.000000,1170.000000,1170.000000,1170.000000,1170.000000,1170.000000,1170.000000,1170.000000,1170.000000,1170.000000,1170.000000,1170.000000,1170.000000,1170.000000,1170.000000
mean,4.050583e+05,41.406164,2.167129,89.911966,2.647863,1.453846,3.494901,0.367251,3.403759,1.847461,0.452137,1.335897,2.361538,0.614530,0.385470,0.737607,0.262393,0.264103,0.735897,0.929060,0.070940,0.400000,0.600000,0.960684,0.039316,0.537607,0.462393,0.929060,0.070940,0.990598,0.009402,0.100000,0.136752,0.050427,0.234188,0.478632,0.610256,0.052137,0.048718,0.210256,0.078632,0.053846,0.004274,0.004274,0.007692,0.004274,0.011966,0.037607,0.876068
std,6.442154e+05,0.025809,0.023118,67.150712,1.130085,0.844652,1.980491,0.295759,2.202354,1.893766,1.109805,1.722640,1.077839,0.486914,0.486914,0.440123,0.440123,0.441043,0.441043,0.256835,0.256835,0.490107,0.490107,0.194429,0.194429,0.498797,0.498797,0.256835,0.256835,0.096547,0.096547,0.300128,0.343732,0.218919,0.423671,0.499757,0.487901,0.222398,0.215370,0.407665,0.269279,0.225810,0.065260,0.065260,0.087405,0.065260,0.108778,0.190325,0.329644
min,2.900000e+04,41.355143,2.099004,10.000000,0.000000,1.000000,0.074264,0.014215,0.000000,0.558659,-1.000000,-1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.396000e+05,41.383035,2.152141,57.000000,2.000000,1.000000,1.782861,0.217491,2.439024,1.388889,-1.000000,-1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,2.025000e+05,41.403535,2.169455,70.000000,3.000000,1.000000,3.415416,0.315010,3.380309,1.666667,1.000000,2.000000,2.000000,1.000000,0.000000,1.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,4.147500e+05,41.424744,2.180327,100.000000,3.000000,2.000000,4.787665,0.429002,4.347826,2.000000,1.000000,2.000000,3.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000


 Number of duplicates: 0


**Tratamiento de Outliers**

Empezaremos por la eliminación de outliers antes de escalar las variables, ya que los outliers pueden afectar la distribución de los datos y hacer que los resultados del escalado no sean los esperados.


Dado que la mayoría de las variables numéricas tienen muchos valores atípicos, usaremos el método IQR para tratar los outliers.

El método IQR es un método robusto para la detección y eliminación de valores atípicos (puede tratar un gran número de valores atípicos en los datos). Además, es menos sensible a los valores extremos que el método Z-score, lo que lo hace más adecuado para un dataset con muchos outliers.

Para utilizar el método IQR, primero calcularemos el IQR para cada variable numérica. A continuación, identificaremos cualquier data point que caiga por debajo de Q1 - 1.5 * IQR o por encima de Q3 + 1.5 * IQR como un valor atípico y lo eliminaremos del conjunto de datos.


In [627]:
def remove_outliers_iqr(df: pd.DataFrame, cols: List[str], threshold: float = 1.5) -> pd.DataFrame:
    """
    Remove outliers from a pandas DataFrame using the interquartile range (IQR) method.
    
    Args:
        df: A pandas DataFrame containing the data.
        cols: A list of columns to remove outliers from.
        threshold: The number of IQRs beyond which a data point is considered an outlier.
                   Defaults to 1.5.
                   
    Returns:
        A pandas DataFrame with the outliers removed.
    """
    # Calculate the IQR for each specified column
    Q1 = df[cols].quantile(0.25)
    Q3 = df[cols].quantile(0.75)
    IQR = Q3 - Q1
    
    # Determine the threshold values for each specified column
    thresholds = (Q1 - threshold * IQR, Q3 + threshold * IQR)
    
    # Identify and remove the outliers
    is_outlier = ((df[cols] < thresholds[0]) | (df[cols] > thresholds[1])).any(axis=1)
    df_clean = df.loc[~is_outlier, :]
    
    return df_clean

In [628]:
# variables numéricas
num_vars = ['price', 'latitude', 'longitude', 
            'sq_meters_built', 'rooms', 'bathrooms', 
            'dist_city_center', 'dist_closest_station',
            'rooms_per_100_sqm', 'bathrooms_per_100_sqm']

In [629]:
# Remove outliers using the IQR method
bagging_df2 = remove_outliers_iqr(bagging_df2, num_vars)
bagging_df2_test = remove_outliers_iqr(bagging_df2_test, num_vars)

In [630]:
# exploración rápida del df
print("TRAIN: ")
print(bagging_df2.shape)
print("TEST: ")
print(bagging_df2_test.shape)

TRAIN: 
(3669, 49)
TEST: 
(898, 49)


**Escalar los atributos**

Escalamos los atributos ya que, por ejemplo, los metros cuadrados o el número de habitaciones, están en una escala diferente a otras características, como la ubicación (longitud y latitud) o el año de construcción. Escalar las características de entrada a una escala común puede ayudar a garantizar que todas estas características reciben el mismo tratamiento en el modelo.

Como los datos no siguen una distribución normal y ya hemos tratado los valores atípicos utilizando la técnica IQR, entonces utilizar el MinMaxScaler puede ser una buena opción para escalar las variables numéricas.

El MinMaxScaler escala los atributos a un rango fijo entre 0 y 1, por lo que es apropiado cuando los atributos no tienen una distribución normal o tienen un rango limitado.

In [631]:
def scale_columns(df: pd.DataFrame, columns: List[str]) -> pd.DataFrame:
    """
    Applies MinMaxScaler to the specified columns of a pandas DataFrame.

    Parameters:
    -----------
    df : pd.DataFrame
        The pandas DataFrame to scale.
    columns : List[str]
        The list of column names to scale.

    Returns:
    --------
    pd.DataFrame
        The scaled pandas DataFrame.
    """

    # Create a copy of the DataFrame
    df_scaled = df.copy()

    # Create a MinMaxScaler object
    scaler = MinMaxScaler()

    # Scale the specified columns
    df_scaled[columns] = scaler.fit_transform(df_scaled[columns])

    return df_scaled


In [632]:
# Scale the columns
bagging_df2 = scale_columns(bagging_df2, num_vars)
bagging_df2_test = scale_columns(bagging_df2_test, num_vars)

El siguiente código hace lo mismo pero usando un Pipeline: 

In [633]:
class OutlierRemover(BaseEstimator, TransformerMixin):
    def __init__(self, cols=None, threshold=1.5):
        self.cols = cols
        self.threshold = threshold

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        X_clean = X.copy()
        # Calculate the IQR for each specified column
        Q1 = X_clean[self.cols].quantile(0.25)
        Q3 = X_clean[self.cols].quantile(0.75)
        IQR = Q3 - Q1

        # Determine the threshold values for each specified column
        thresholds = (Q1 - self.threshold * IQR, Q3 + self.threshold * IQR)

        # Identify and remove the outliers
        is_outlier = ((X_clean[self.cols] < thresholds[0]) | (X_clean[self.cols] > thresholds[1])).any(axis=1)
        X_clean = X_clean.loc[~is_outlier, :]

        return X_clean


class MinMaxScalerTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.scaler = MinMaxScaler()

    def fit(self, X, y=None):
        self.scaler.fit(X)
        return self

    def transform(self, X):
        X_scaled = self.scaler.transform(X)
        feature_names = [f'{col}' for col in X.columns]
        return pd.DataFrame(X_scaled, columns=feature_names)


In [1248]:
# define the preprocessing pipeline
preprocessing_pipeline = Pipeline([
    ('outlier_removal', OutlierRemover(cols=num_vars)),
    ('scaler', ColumnTransformer([('num_scaler', MinMaxScaler(), num_vars)], 
                                 remainder='passthrough'))
])


# Apply the preprocessing pipeline to the training and testing data
    # Fit and transform the training data
bagging_df2 = pd.DataFrame(preprocessing_pipeline.fit_transform(bagging_df2), columns=num_vars + list(bagging_df2.columns[len(num_vars):]))
bagging_df2_test = pd.DataFrame(preprocessing_pipeline.fit_transform(bagging_df2_test), columns=num_vars + list(bagging_df2_test.columns[len(num_vars):]))

##### Model 2: Bagging & RF 

Una vez hemos tratado los outliers y hemos escalado los atributos numéricos, vemos cómo se comportan los modelos:

In [635]:
# create an instance of PipelineRunner
pipeline_runner = PipelineRunner()
# call the run_pipeline() method, passing in the dataframes and target variable name
results_2 = pipeline_runner.run_pipeline(df_train=bagging_df2, 
                                         df_test=bagging_df2_test, 
                                         target=TARGET_VARIABLE)
# examine the results for each model
results_2

Model: BaggingRegressor
Best model parameters: {'model__n_estimators': 200}
Best cross-validation R^2 score: 0.7708
Train R^2 score: 0.9701
Test R^2 score: 0.6908

Model: RandomForestRegressor
Best model parameters: {'model__max_depth': 9, 'model__min_samples_split': 2, 'model__n_estimators': 200}
Best cross-validation R^2 score: 0.7601
Train R^2 score: 0.9245
Test R^2 score: 0.6744

Best model: BaggingRegressor
Best model parameters: {'model__n_estimators': 200}
Best cross-validation R^2 score: 0.7708
Train R^2 score: 0.9701
Test R^2 score: 0.6908


[{'name': 'BaggingRegressor',
  'best_params': {'model__n_estimators': 200},
  'best_score': 0.7707652188812547,
  'train_r2_score': 0.97011092170689,
  'test_r2_score': 0.6907957655585963},
 {'name': 'RandomForestRegressor',
  'best_params': {'model__max_depth': 9,
   'model__min_samples_split': 2,
   'model__n_estimators': 200},
  'best_score': 0.7600922045988325,
  'train_r2_score': 0.9245244491203484,
  'test_r2_score': 0.6744430145609775}]

In [656]:
# Loop through each model in results
for model in results_2:
    model_name = model['name']+'_2'
    train_r2_score = model['train_r2_score']
    test_r2_score = model['test_r2_score']
    
    # Append the results to the DataFrame
    models_results_df = pd.concat([models_results_df, pd.DataFrame({'regressor': model_name, 
                                                                   'train_r2_score': train_r2_score,
                                                                   'test_r2_score': test_r2_score},
                                                                   index=[0])], ignore_index=True)
# Display the DataFrame
display(models_results_df)

,regressor,train_r2_score,test_r2_score
0,BaggingRegressor_1,0.976685,0.739290
1,RandomForestRegressor_1,0.939248,0.723103
2,BaggingRegressor_2,0.970111,0.690796
3,RandomForestRegressor_2,0.924524,0.674443


- Parece que de momento, el tratamiento de outliers y el escalado de las variables numéricas no han mejorado el rendimiento de los modelos, sino todo lo contrario. 



#### Modelo 3

Recordemos lo que obtuvimos de cada técnica (para seleccionar las mejores variables predictoras del precio de la vivienda): 

- Del análisis de componentes principales (PCA), observamos que las variables sq_meters_built, bathrooms, longitude, dist_city_center, latitude, bathrooms_per_100_sqm y rooms_per_100_sqm tenían los mayores pesos en los primeros 3 componentes principales, lo que indica que estas variables contribuyen significativamente a la variación total de los datos. Sin embargo, se requiere un análisis adicional para determinar si estas variables tienen una relación significativa con el precio de la vivienda y cómo se relacionan con él.

- Del análisis de información mutua (recordemos que una puntuación más alta significa que la característica es más informativa para predecir la variable objetivo) vimos que "sq_meters_built" tenía la puntuación de información mutua más alta, lo que significa que puede ser el atributo más informativo para predecir el precio de la vivienda. Del mismo modo, "rooms_per_100_sqm" y "bathrooms_per_100_sqm" también tenían puntuaciones de información mutua relativamente altas, lo que indica que también son atributos informativos. Otros atributos que tienen puntuaciones de información mutua relativamente altas eran " latitude", "bathrooms" y "neighborhood_rent_index_cluster ", mientras que atributos como "property_type_duplex", "property_type_chalet_adosado" y "heating_electric" tenían puntuaciones muy bajas, lo que indica que son menos informativos a la hora predecir el precio de la vivienda.

- Del análisis ANOVA (y Kruskall-Wallis) vimos que las variables con mayor efecto sobre el precio de venta de la vivienda eran neighborhood_rent_index_cluster, garage, garden, pool, terrace, property_type, property_age_cat, elevator, ac, heating y property_age_cat.

- Al realizar el método ExtraTreesRegressor vimos que que según el modelo, sq_meters_built es el atributo más importante para predecir el precio de la vivienda, seguida de bathrooms y neighborhood_rent_index_cluster_encoded. Otras características como rooms_per_100_sqm, garden y bathrooms_per_100_sqm también tenían puntuaciones de importancia relativamente altas.


Al considerar los resultados de los diferentes métodos, parece que "sq_meters_built", "bathrooms", "longitude", "dist_city_center", "latitude", "bathrooms_per_100_sqm", "rooms_per_100_sqm", "neighborhood_rent_index_cluster", "garage", "garden", "pool", "terrace", "property_type", "property_age_cat", "elevator", "ac", "heating" y "property_age_cat" son las variables más importantes o informativas para predecir el precio de la vivienda en Barcelona. Recordemos que "bathrooms" estaba fuertemente correlacionado con "sq_meters_built" y por eso creamos "bathrooms_per_100_sqm". Por tanto, nos quedamos con "bathrooms_per_100_sqm". 

Por tanto, realizaremos los modelos de Bagging y Random Forest con estas variables. 
Como los resultados han mejorado después de tratar los outliers y escalar los atributos numéricos, relizaremos el mismo proceso con las variables seleccionadas. 

#####  Train dataset (bagging_df3) y Test dataset (bagging_df3_test)

In [636]:
# creamos los datasets para el tercer modelo de Bagging y RF
bagging_df3 = sales_simplified_df.copy(deep=True)
bagging_df3_test = test_data_preprocessed.copy(deep=True)

# lista contiene las variables que se consideran más importantes o informativas 
# para predecir el precio de la vivienda en Barcelona, según los diferentes métodos de análisis utilizados.
good_predictors = ["price", # target variable
                   "sq_meters_built", "longitude", 
                   "dist_city_center", "latitude", "bathrooms_per_100_sqm", 
                   "rooms_per_100_sqm", "neighborhood_rent_index_cluster",                    
                   "garage", "garden", "pool", "terrace", "property_type", 
                   "property_age_cat", "elevator", "ac", "heating"]
# nos quedamos sólo con las columnas indicadas en la lista good_predictors
bagging_df3 = bagging_df3[good_predictors]
bagging_df3_test = bagging_df3_test[good_predictors]

# variables que queremos codificar usando ordinal encoder
oe_cols = oe_cols = ['property_age_cat', 'neighborhood_rent_index_cluster']
# Define the order of the categories for each column
property_age_cat_order = ['Unknown', '0 - 40', '40 - 70', '70 - 120', '120 - 150', '+150']
neighborhood_rent_index_cluster_order = ['Unknown', '0 - 2337', '2337 - 3371', '3371 - 5042', '+5042']
# Create an instance of the ordinal encoder
encoder = OrdinalEncoder(categories=[property_age_cat_order, 
                                     neighborhood_rent_index_cluster_order])

for col in oe_cols:
    # Create a new column name for the encoded data
    new_col = col + '_encoded'
    # Create an instance of the ordinal encoder
    encoder = OrdinalEncoder(categories=[eval(col+'_order')])
    # Fit and transform the selected columns
    bagging_df3[new_col] = encoder.fit_transform(bagging_df3[[col]])
    bagging_df3_test[new_col] = encoder.fit_transform(bagging_df3_test[[col]])
    # replace 0 by -1 ('Unknown' by -1)
    bagging_df3[new_col] = bagging_df3[new_col].replace(0, -1)
    bagging_df3_test[new_col] = bagging_df3_test[new_col].replace(0, -1)

# variables que queremos codificar usando OHE
ohe_cols = ["garage", "garden", "pool", 
            "terrace", "property_type", 
            "elevator", "ac", "heating"]

# aplicamos one - hot encoding al dataset
bagging_df3 = one_hot_encode(bagging_df3, ohe_cols)
bagging_df3_test = one_hot_encode(bagging_df3_test, ohe_cols)

# eliminamos variables categóricas no codificadas
no_encoded_cat_vars = bagging_df3.select_dtypes(include=['category']).columns.tolist()
bagging_df3.drop(no_encoded_cat_vars, axis=1, inplace=True)
no_encoded_cat_vars_test = bagging_df3_test.select_dtypes(include=['category']).columns.tolist()
bagging_df3_test.drop(no_encoded_cat_vars_test, axis=1, inplace=True)


############# Tratamos los outliers y escalamos los atributos numéricos: ###############
num_vars = ["price", "sq_meters_built", "longitude", "dist_city_center", "latitude", "bathrooms_per_100_sqm", "rooms_per_100_sqm"]

# define the preprocessing pipeline
preprocessing_pipeline = Pipeline([
    ('outlier_removal', OutlierRemover(cols=num_vars)),
    ('scaler', ColumnTransformer([('num_scaler', MinMaxScaler(), num_vars)], 
                                 remainder='passthrough'))
])

# Apply the preprocessing pipeline to the training and testing data
    # Fit and transform the training data
bagging_df3 = pd.DataFrame(preprocessing_pipeline.fit_transform(bagging_df3), columns=num_vars + list(bagging_df3.columns[len(num_vars):]))
bagging_df3_test = pd.DataFrame(preprocessing_pipeline.fit_transform(bagging_df3_test), columns=num_vars + list(bagging_df3_test.columns[len(num_vars):]))

# exploración rápida del df
print("TRAIN: ")
explore_data(bagging_df3)
print("\n\n")
print("TEST: ")
explore_data(bagging_df3_test)

TRAIN: 
The dataset includes 3914 instances (rows) and 34 variables (columns).



`sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
`sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.


,price,sq_meters_built,longitude,dist_city_center,latitude,bathrooms_per_100_sqm,rooms_per_100_sqm,property_age_cat_encoded,neighborhood_rent_index_cluster_encoded,garage_0,garage_1,garden_0,garden_1,pool_0,pool_1,terrace_0,terrace_1,property_type_atico,property_type_chalet,property_type_chalet_adosado,property_type_chalet_independiente,property_type_chalet_pareado,property_type_duplex,property_type_estudio,property_type_piso,elevator_0,elevator_1,ac_0,ac_1,heating_Unknown,heating_bomba,heating_electric,heating_gas,heating_individual
0,0.083558,0.149123,0.539966,0.598009,0.721144,0.555140,0.280000,2.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
1,0.157682,0.175439,0.578853,0.729485,0.811563,0.506260,0.792453,-1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
2,0.871968,0.763158,0.375652,0.162188,0.337326,0.411215,0.466667,2.0,3.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
3,0.392183,0.315789,0.457874,0.331045,0.529310,0.317351,0.608696,3.0,3.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
4,0.080863,0.271930,0.391947,0.470643,0.616941,0.366238,0.437500,2.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0




<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3914 entries, 0 to 3913
Data columns (total 34 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   price                                    3914 non-null   float64
 1   sq_meters_built                          3914 non-null   float64
 2   longitude                                3914 non-null   float64
 3   dist_city_center                         3914 non-null   float64
 4   latitude                                 3914 non-null   float64
 5   bathrooms_per_100_sqm                    3914 non-null   float64
 6   rooms_per_100_sqm                        3914 non-null   float64
 7   property_age_cat_encoded                 3914 non-null   float64
 8   neighborhood_rent_index_cluster_encoded  3914 non-null   float64
 9   garage_0                                 3914 non-null   float64
 10  garage_1                                 3914 

,price,sq_meters_built,longitude,dist_city_center,latitude,bathrooms_per_100_sqm,rooms_per_100_sqm,property_age_cat_encoded,neighborhood_rent_index_cluster_encoded,garage_0,garage_1,garden_0,garden_1,pool_0,pool_1,terrace_0,terrace_1,property_type_atico,property_type_chalet,property_type_chalet_adosado,property_type_chalet_independiente,property_type_chalet_pareado,property_type_duplex,property_type_estudio,property_type_piso,elevator_0,elevator_1,ac_0,ac_1,heating_Unknown,heating_bomba,heating_electric,heating_gas,heating_individual
count,3914.000000,3914.000000,3914.000000,3914.000000,3914.000000,3914.000000,3914.000000,3914.000000,3914.000000,3914.000000,3914.000000,3914.000000,3914.000000,3914.000000,3914.000000,3914.000000,3914.000000,3914.000000,3914.000000,3914.000000,3914.000000,3914.000000,3914.000000,3914.000000,3914.000000,3914.000000,3914.000000,3914.000000,3914.000000,3914.000000,3914.000000,3914.000000,3914.000000,3914.000000
mean,0.278795,0.327542,0.534793,0.424337,0.488343,0.460803,0.501716,1.296883,2.045989,0.970363,0.029637,0.999234,0.000766,0.991824,0.008176,0.811957,0.188043,0.041901,0.001022,0.002044,0.001022,0.002044,0.013797,0.028871,0.909300,0.457333,0.542667,0.591722,0.408278,0.616505,0.059019,0.040112,0.213337,0.071027
std,0.195855,0.197310,0.200325,0.240007,0.247701,0.200167,0.174327,1.716043,0.983602,0.169606,0.169606,0.027678,0.027678,0.090061,0.090061,0.390796,0.390796,0.200388,0.031956,0.045170,0.031956,0.045170,0.116661,0.167465,0.287219,0.498240,0.498240,0.491578,0.491578,0.486299,0.235690,0.196248,0.409716,0.256903
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.143026,0.192982,0.401951,0.212459,0.245523,0.317351,0.394366,-1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.176550,0.289474,0.550578,0.418645,0.495922,0.411215,0.525000,2.000000,2.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
75%,0.386456,0.421053,0.642030,0.585306,0.684464,0.572764,0.622222,2.000000,3.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,5.000000,4.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000



 Number of duplicates: 3



TEST: 
The dataset includes 970 instances (rows) and 34 variables (columns).



,price,sq_meters_built,longitude,dist_city_center,latitude,bathrooms_per_100_sqm,rooms_per_100_sqm,property_age_cat_encoded,neighborhood_rent_index_cluster_encoded,garage_0,garage_1,garden_0,garden_1,pool_0,pool_1,terrace_0,terrace_1,property_type_atico,property_type_chalet,property_type_chalet_adosado,property_type_chalet_independiente,property_type_chalet_pareado,property_type_duplex,property_type_estudio,property_type_piso,elevator_0,elevator_1,ac_0,ac_1,heating_Unknown,heating_bomba,heating_electric,heating_gas,heating_individual
0,0.130653,0.320312,0.934009,0.601814,0.603811,0.292241,0.552632,2.0,2.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.572864,0.742188,0.379277,0.187371,0.234849,0.391812,0.430769,3.0,3.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
2,0.296482,0.328125,0.663388,0.274098,0.473093,0.284600,0.545455,2.0,3.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
3,0.157663,0.203125,0.560432,0.597186,0.721143,0.436924,0.459016,2.0,2.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.134422,0.257812,0.561850,0.200849,0.155197,0.361462,0.617647,2.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0




<class 'pandas.core.frame.DataFrame'>
RangeIndex: 970 entries, 0 to 969
Data columns (total 34 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   price                                    970 non-null    float64
 1   sq_meters_built                          970 non-null    float64
 2   longitude                                970 non-null    float64
 3   dist_city_center                         970 non-null    float64
 4   latitude                                 970 non-null    float64
 5   bathrooms_per_100_sqm                    970 non-null    float64
 6   rooms_per_100_sqm                        970 non-null    float64
 7   property_age_cat_encoded                 970 non-null    float64
 8   neighborhood_rent_index_cluster_encoded  970 non-null    float64
 9   garage_0                                 970 non-null    float64
 10  garage_1                                 970 non

,price,sq_meters_built,longitude,dist_city_center,latitude,bathrooms_per_100_sqm,rooms_per_100_sqm,property_age_cat_encoded,neighborhood_rent_index_cluster_encoded,garage_0,garage_1,garden_0,garden_1,pool_0,pool_1,terrace_0,terrace_1,property_type_atico,property_type_chalet,property_type_chalet_adosado,property_type_chalet_independiente,property_type_chalet_pareado,property_type_duplex,property_type_estudio,property_type_piso,elevator_0,elevator_1,ac_0,ac_1,heating_Unknown,heating_bomba,heating_electric,heating_gas,heating_individual
count,970.000000,970.000000,970.000000,970.000000,970.000000,970.000000,970.000000,970.000000,970.000000,970.000000,970.000000,970.000000,970.000000,970.000000,970.000000,970.000000,970.000000,970.000000,970.000000,970.000000,970.000000,970.000000,970.000000,970.000000,970.000000,970.000000,970.000000,970.000000,970.000000,970.000000,970.000000,970.000000,970.000000,970.000000
mean,0.278775,0.306258,0.530647,0.429441,0.488279,0.476849,0.498464,1.288660,2.182474,0.968041,0.031959,0.997938,0.002062,0.986598,0.013402,0.790722,0.209278,0.041237,0.002062,0.001031,0.001031,0.002062,0.010309,0.030928,0.911340,0.415464,0.584536,0.573196,0.426804,0.637113,0.054639,0.046392,0.194845,0.067010
std,0.204398,0.199326,0.200919,0.234314,0.247501,0.194325,0.174288,1.694152,1.017241,0.175981,0.175981,0.045384,0.045384,0.115048,0.115048,0.407003,0.407003,0.198941,0.045384,0.032108,0.032108,0.045384,0.101062,0.173212,0.284399,0.493056,0.493056,0.494869,0.494869,0.481081,0.227392,0.210441,0.396286,0.250169
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.137563,0.171875,0.404450,0.228346,0.250443,0.342673,0.388889,-1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.184673,0.257812,0.550297,0.422272,0.496598,0.449142,0.512195,2.000000,2.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
75%,0.376884,0.398438,0.643505,0.583830,0.675860,0.598197,0.617647,2.000000,3.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,5.000000,4.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000



 Number of duplicates: 0


##### Model 3: Bagging & RF 

Una vez hemos hemos seleccionado las variables predictoras más importantes, hemos tratado los outliers y hemos escalado los atributos numéricos, vemos cómo se comportan los modelos:

In [1294]:
# create an instance of PipelineRunner
pipeline_runner = PipelineRunner()
# call the run_pipeline() method, passing in the dataframes and target variable name
results_3 = pipeline_runner.run_pipeline(df_train=bagging_df3, 
                                         df_test=bagging_df3_test, 
                                         target=TARGET_VARIABLE)
# examine the results for each model
results_3

[{'model_name': 'BaggingRegressor',
  'best_params': {'model__n_estimators': 200},
  'train_r2_score': 0.982214048578808,
  'test_r2_score': 0.8512093886379279},
 {'model_name': 'RandomForestRegressor',
  'best_params': {'model__max_depth': 9,
   'model__min_samples_split': 2,
   'model__n_estimators': 200},
  'train_r2_score': 0.9479322101359778,
  'test_r2_score': 0.8462529129408495}]

In [637]:
# create an instance of PipelineRunner
pipeline_runner = PipelineRunner()
# call the run_pipeline() method, passing in the dataframes and target variable name
results_3 = pipeline_runner.run_pipeline(df_train=bagging_df3, 
                                         df_test=bagging_df3_test, 
                                         target=TARGET_VARIABLE)
# examine the results for each model
results_3

Model: BaggingRegressor
Best model parameters: {'model__n_estimators': 200}
Best cross-validation R^2 score: 0.8708
Train R^2 score: 0.9823
Test R^2 score: 0.8525

Model: RandomForestRegressor
Best model parameters: {'model__max_depth': 9, 'model__min_samples_split': 2, 'model__n_estimators': 100}
Best cross-validation R^2 score: 0.8618
Train R^2 score: 0.9469
Test R^2 score: 0.8451

Best model: BaggingRegressor
Best model parameters: {'model__n_estimators': 200}
Best cross-validation R^2 score: 0.8708
Train R^2 score: 0.9823
Test R^2 score: 0.8525


[{'name': 'BaggingRegressor',
  'best_params': {'model__n_estimators': 200},
  'best_score': 0.8707953896715244,
  'train_r2_score': 0.9822737024925415,
  'test_r2_score': 0.8525076454575563},
 {'name': 'RandomForestRegressor',
  'best_params': {'model__max_depth': 9,
   'model__min_samples_split': 2,
   'model__n_estimators': 100},
  'best_score': 0.8618490056134946,
  'train_r2_score': 0.9469431030838147,
  'test_r2_score': 0.8451119639621244}]

In [657]:
# Loop through each model in results
for model in results_3:
    model_name = model['name']+'_3'
    train_r2_score = model['train_r2_score']
    test_r2_score = model['test_r2_score']
    
    # Append the results to the DataFrame
    models_results_df = pd.concat([models_results_df, pd.DataFrame({'regressor': model_name, 
                                                                   'train_r2_score': train_r2_score,
                                                                   'test_r2_score': test_r2_score},
                                                                   index=[0])], ignore_index=True)
# Display the DataFrame
display(models_results_df)

,regressor,train_r2_score,test_r2_score
0,BaggingRegressor_1,0.976685,0.739290
1,RandomForestRegressor_1,0.939248,0.723103
2,BaggingRegressor_2,0.970111,0.690796
3,RandomForestRegressor_2,0.924524,0.674443
4,BaggingRegressor_3,0.982274,0.852508
5,RandomForestRegressor_3,0.946943,0.845112


- Seleccionando las variables predictoras más importantes los modelos han funcionado mucho mejor.

##### Model 3.1: Bagging & RF (NO SCALING)

Escalar los atributos numéricos puede dificultar la interpretación de los resultados. Por tanto, probaremos el modelo anterior (Modelo 3) sin escalar los atributos numéricos. Si nos dá un resultado parecido en validación (o la diferencia no es significativa), nos quedaremos con el modelo sin estandarizar las variables, ya que será más fácil de interpertar. 

In [428]:
# creamos los datasets para el tercer modelo de Bagging y RF
bagging_df3 = sales_simplified_df.copy(deep=True)
bagging_df3_test = test_data_preprocessed.copy(deep=True)

# lista contiene las variables que se consideran más importantes o informativas 
# para predecir el precio de la vivienda en Barcelona, según los diferentes métodos de análisis utilizados.
good_predictors = ["price", # target variable
                   "sq_meters_built", "longitude", 
                   "dist_city_center", "latitude", "bathrooms_per_100_sqm", 
                   "rooms_per_100_sqm", "neighborhood_rent_index_cluster",                    
                   "garage", "garden", "pool", "terrace", "property_type", 
                   "property_age_cat", "elevator", "ac", "heating"]
# nos quedamos sólo con las columnas indicadas en la lista good_predictors
bagging_df3 = bagging_df3[good_predictors]
bagging_df3_test = bagging_df3_test[good_predictors]

# variables que queremos codificar usando ordinal encoder
oe_cols = oe_cols = ['property_age_cat', 'neighborhood_rent_index_cluster']
# Define the order of the categories for each column
property_age_cat_order = ['Unknown', '0 - 40', '40 - 70', '70 - 120', '120 - 150', '+150']
neighborhood_rent_index_cluster_order = ['Unknown', '0 - 2337', '2337 - 3371', '3371 - 5042', '+5042']
# Create an instance of the ordinal encoder
encoder = OrdinalEncoder(categories=[property_age_cat_order, 
                                     neighborhood_rent_index_cluster_order])

for col in oe_cols:
    # Create a new column name for the encoded data
    new_col = col + '_encoded'
    # Create an instance of the ordinal encoder
    encoder = OrdinalEncoder(categories=[eval(col+'_order')])
    # Fit and transform the selected columns
    bagging_df3[new_col] = encoder.fit_transform(bagging_df3[[col]])
    bagging_df3_test[new_col] = encoder.fit_transform(bagging_df3_test[[col]])
    # replace 0 by -1 ('Unknown' by -1)
    bagging_df3[new_col] = bagging_df3[new_col].replace(0, -1)
    bagging_df3_test[new_col] = bagging_df3_test[new_col].replace(0, -1)

# variables que queremos codificar usando OHE
ohe_cols = ["garage", "garden", "pool", 
            "terrace", "property_type", 
            "elevator", "ac", "heating"]

# aplicamos one - hot encoding al dataset
bagging_df3 = one_hot_encode(bagging_df3, ohe_cols)
bagging_df3_test = one_hot_encode(bagging_df3_test, ohe_cols)

# eliminamos variables categóricas no codificadas
no_encoded_cat_vars = bagging_df3.select_dtypes(include=['category']).columns.tolist()
bagging_df3.drop(no_encoded_cat_vars, axis=1, inplace=True)
no_encoded_cat_vars_test = bagging_df3_test.select_dtypes(include=['category']).columns.tolist()
bagging_df3_test.drop(no_encoded_cat_vars_test, axis=1, inplace=True)


############# Tratamos los outliers per NO escalamos los atributos numéricos: ###############
num_vars = ["price", "sq_meters_built", "longitude", "dist_city_center", "latitude", "bathrooms_per_100_sqm", "rooms_per_100_sqm"]

# define the preprocessing pipeline
preprocessing_pipeline = Pipeline([
    ('outlier_removal', OutlierRemover(cols=num_vars)),
    #('scaler', ColumnTransformer([('num_scaler', MinMaxScaler(), num_vars)], 
                                # remainder='passthrough'))
])

# Apply the preprocessing pipeline to the training and testing data
    # Fit and transform the training data
bagging_df3 = pd.DataFrame(preprocessing_pipeline.fit_transform(bagging_df3), columns=num_vars + list(bagging_df3.columns[len(num_vars):]))
bagging_df3_test = pd.DataFrame(preprocessing_pipeline.fit_transform(bagging_df3_test), columns=num_vars + list(bagging_df3_test.columns[len(num_vars):]))

# exploración rápida del df
print("TRAIN: ")
explore_data(bagging_df3)
print("\n\n")
print("TEST: ")
explore_data(bagging_df3_test)

TRAIN: 
The dataset includes 3914 instances (rows) and 34 variables (columns).



`sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
`sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.


,price,sq_meters_built,longitude,dist_city_center,latitude,bathrooms_per_100_sqm,rooms_per_100_sqm,property_age_cat_encoded,neighborhood_rent_index_cluster_encoded,garage_0,garage_1,garden_0,garden_1,pool_0,pool_1,terrace_0,terrace_1,property_type_atico,property_type_chalet,property_type_chalet_adosado,property_type_chalet_independiente,property_type_chalet_pareado,property_type_duplex,property_type_estudio,property_type_piso,elevator_0,elevator_1,ac_0,ac_1,heating_Unknown,heating_bomba,heating_electric,heating_gas,heating_individual
1,90000,50,2.171110,5.189265,41.434545,2.000000,2.000000,2.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
2,145000,53,2.175309,6.313668,41.444518,1.886792,5.660377,-1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
3,675000,120,2.153368,1.462060,41.392209,1.666667,3.333333,2.0,3.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
4,319000,69,2.162246,2.906147,41.413385,1.449275,4.347826,3.0,3.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
5,88000,64,2.155127,4.100012,41.423051,1.562500,3.125000,2.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0




<class 'pandas.core.frame.DataFrame'>
Int64Index: 3914 entries, 1 to 4673
Data columns (total 34 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   price                                    3914 non-null   int64  
 1   sq_meters_built                          3914 non-null   int64  
 2   longitude                                3914 non-null   float64
 3   dist_city_center                         3914 non-null   float64
 4   latitude                                 3914 non-null   float64
 5   bathrooms_per_100_sqm                    3914 non-null   float64
 6   rooms_per_100_sqm                        3914 non-null   float64
 7   property_age_cat_encoded                 3914 non-null   float64
 8   neighborhood_rent_index_cluster_encoded  3914 non-null   float64
 9   garage_0                                 3914 non-null   float64
 10  garage_1                                 3914 

,price,sq_meters_built,longitude,dist_city_center,latitude,bathrooms_per_100_sqm,rooms_per_100_sqm,property_age_cat_encoded,neighborhood_rent_index_cluster_encoded,garage_0,garage_1,garden_0,garden_1,pool_0,pool_1,terrace_0,terrace_1,property_type_atico,property_type_chalet,property_type_chalet_adosado,property_type_chalet_independiente,property_type_chalet_pareado,property_type_duplex,property_type_estudio,property_type_piso,elevator_0,elevator_1,ac_0,ac_1,heating_Unknown,heating_bomba,heating_electric,heating_gas,heating_individual
count,3914.000000,3914.000000,3914.000000,3914.000000,3914.000000,3914.000000,3914.000000,3914.000000,3914.000000,3914.000000,3914.000000,3914.000000,3914.000000,3914.000000,3914.000000,3914.000000,3914.000000,3914.000000,3914.000000,3914.000000,3914.000000,3914.000000,3914.000000,3914.000000,3914.000000,3914.000000,3914.000000,3914.000000,3914.000000,3914.000000,3914.000000,3914.000000,3914.000000,3914.000000
mean,234866.117016,70.339806,2.170552,3.703994,41.408867,1.781512,3.583683,1.296883,2.045989,0.970363,0.029637,0.999234,0.000766,0.991824,0.008176,0.811957,0.188043,0.041901,0.001022,0.002044,0.001022,0.002044,0.013797,0.028871,0.909300,0.457333,0.542667,0.591722,0.408278,0.616505,0.059019,0.040112,0.213337,0.071027
std,145324.083035,22.493365,0.021631,2.052572,0.027322,0.463591,1.245189,1.716043,0.983602,0.169606,0.169606,0.027678,0.027678,0.090061,0.090061,0.390796,0.390796,0.200388,0.031956,0.045170,0.031956,0.045170,0.116661,0.167465,0.287219,0.498240,0.498240,0.491578,0.491578,0.486299,0.235690,0.196248,0.409716,0.256903
min,28000.000000,33.000000,2.112805,0.075001,41.355002,0.714286,0.000000,-1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,134125.000000,55.000000,2.156207,1.891978,41.382083,1.449275,2.816901,-1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,159000.000000,66.000000,2.172256,3.655316,41.409703,1.666667,3.750000,2.000000,2.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
75%,314750.000000,81.000000,2.182131,5.080628,41.430499,2.040816,4.444444,2.000000,3.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000
max,770000.000000,147.000000,2.220785,8.627149,41.465303,3.030303,7.142857,5.000000,4.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000



 Number of duplicates: 3



TEST: 
The dataset includes 970 instances (rows) and 34 variables (columns).



,price,sq_meters_built,longitude,dist_city_center,latitude,bathrooms_per_100_sqm,rooms_per_100_sqm,property_age_cat_encoded,neighborhood_rent_index_cluster_encoded,garage_0,garage_1,garden_0,garden_1,pool_0,pool_1,terrace_0,terrace_1,property_type_atico,property_type_chalet,property_type_chalet_adosado,property_type_chalet_independiente,property_type_chalet_pareado,property_type_duplex,property_type_estudio,property_type_piso,elevator_0,elevator_1,ac_0,ac_1,heating_Unknown,heating_bomba,heating_electric,heating_gas,heating_individual
0,133000,76,2.213120,5.158288,41.421077,1.315789,3.947368,2.0,2.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,485000,130,2.152472,1.657143,41.380788,1.538462,3.076923,3.0,3.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
2,265000,77,2.183533,2.389801,41.406803,1.298701,3.896104,2.0,3.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
3,154500,61,2.172277,5.119194,41.433890,1.639344,3.278689,2.0,2.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
6,136000,68,2.172432,1.771000,41.372090,1.470588,4.411765,2.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0




<class 'pandas.core.frame.DataFrame'>
Int64Index: 970 entries, 0 to 1169
Data columns (total 34 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   price                                    970 non-null    int64  
 1   sq_meters_built                          970 non-null    int64  
 2   longitude                                970 non-null    float64
 3   dist_city_center                         970 non-null    float64
 4   latitude                                 970 non-null    float64
 5   bathrooms_per_100_sqm                    970 non-null    float64
 6   rooms_per_100_sqm                        970 non-null    float64
 7   property_age_cat_encoded                 970 non-null    float64
 8   neighborhood_rent_index_cluster_encoded  970 non-null    float64
 9   garage_0                                 970 non-null    float64
 10  garage_1                                 970 no

,price,sq_meters_built,longitude,dist_city_center,latitude,bathrooms_per_100_sqm,rooms_per_100_sqm,property_age_cat_encoded,neighborhood_rent_index_cluster_encoded,garage_0,garage_1,garden_0,garden_1,pool_0,pool_1,terrace_0,terrace_1,property_type_atico,property_type_chalet,property_type_chalet_adosado,property_type_chalet_independiente,property_type_chalet_pareado,property_type_duplex,property_type_estudio,property_type_piso,elevator_0,elevator_1,ac_0,ac_1,heating_Unknown,heating_bomba,heating_electric,heating_gas,heating_individual
count,970.000000,970.000000,970.000000,970.000000,970.000000,970.000000,970.000000,970.000000,970.000000,970.000000,970.000000,970.000000,970.000000,970.000000,970.000000,970.000000,970.000000,970.000000,970.000000,970.000000,970.000000,970.000000,970.000000,970.000000,970.000000,970.000000,970.000000,970.000000,970.000000,970.000000,970.000000,970.000000,970.000000,970.000000
mean,250904.920619,74.201031,2.169021,3.702115,41.408462,1.728628,3.560459,1.288660,2.182474,0.968041,0.031959,0.997938,0.002062,0.986598,0.013402,0.790722,0.209278,0.041237,0.002062,0.001031,0.001031,0.002062,0.010309,0.030928,0.911340,0.415464,0.584536,0.573196,0.426804,0.637113,0.054639,0.046392,0.194845,0.067010
std,162700.963515,25.513666,0.021966,1.979443,0.027026,0.434568,1.244912,1.694152,1.017241,0.175981,0.175981,0.045384,0.045384,0.115048,0.115048,0.407003,0.407003,0.198941,0.045384,0.032108,0.032108,0.045384,0.101062,0.173212,0.284399,0.493056,0.493056,0.494869,0.494869,0.481081,0.227392,0.210441,0.396286,0.250169
min,29000.000000,35.000000,2.111006,0.074264,41.355143,0.662252,0.000000,-1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,138500.000000,57.000000,2.155224,2.003293,41.382491,1.428571,2.777778,-1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,176000.000000,68.000000,2.171169,3.641553,41.409370,1.666667,3.658537,2.000000,2.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
75%,329000.000000,86.000000,2.181359,5.006365,41.428945,2.000000,4.411765,2.000000,3.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000
max,825000.000000,163.000000,2.220334,8.522102,41.464340,2.898551,7.142857,5.000000,4.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000



 Number of duplicates: 0


In [512]:
# create an instance of PipelineRunner
pipeline_runner = PipelineRunner()  
# call the run_pipeline() method, passing in the dataframes and target variable name
results_4 = pipeline_runner.run_pipeline(df_train=bagging_df3, 
                                         df_test=bagging_df3_test, 
                                         target=TARGET_VARIABLE)


Model: BaggingRegressor
Best model parameters: {'model__n_estimators': 200}
Best cross-validation R^2 score: 0.8699
Train R^2 score: 0.9826
Test R^2 score: 0.8562

Model: RandomForestRegressor
Best model parameters: {'model__max_depth': 9, 'model__min_samples_split': 2, 'model__n_estimators': 100}
Best cross-validation R^2 score: 0.8615
Train R^2 score: 0.9473
Test R^2 score: 0.8466

Best model: BaggingRegressor
Best model parameters: {'model__n_estimators': 200}
Best cross-validation R^2 score: 0.8699
Train R^2 score: 0.9826
Test R^2 score: 0.8562


In [513]:
results_4

[{'name': 'BaggingRegressor',
  'best_params': {'model__n_estimators': 200},
  'best_score': 0.8699084485798494,
  'train_r2_score': 0.9826165282700704,
  'test_r2_score': 0.8562174003334514},
 {'name': 'RandomForestRegressor',
  'best_params': {'model__max_depth': 9,
   'model__min_samples_split': 2,
   'model__n_estimators': 100},
  'best_score': 0.8614809042234631,
  'train_r2_score': 0.9472748317785578,
  'test_r2_score': 0.8466012770286642}]

In [658]:
# Loop through each model in results
for model in results_4:
    model_name = model['name']+'_4'
    train_r2_score = model['train_r2_score']
    test_r2_score = model['test_r2_score']
    
    # Append the results to the DataFrame
    models_results_df = pd.concat([models_results_df, pd.DataFrame({'regressor': model_name, 
                                                                   'train_r2_score': train_r2_score,
                                                                   'test_r2_score': test_r2_score},
                                                                   index=[0])], ignore_index=True)
# Display the DataFrame
display(models_results_df)

,regressor,train_r2_score,test_r2_score
0,BaggingRegressor_1,0.976685,0.739290
1,RandomForestRegressor_1,0.939248,0.723103
2,BaggingRegressor_2,0.970111,0.690796
3,RandomForestRegressor_2,0.924524,0.674443
4,BaggingRegressor_3,0.982274,0.852508
5,RandomForestRegressor_3,0.946943,0.845112
6,BaggingRegressor_4,0.982617,0.856217
7,RandomForestRegressor_4,0.947275,0.846601


- Observamos que sin escalar los atributos numéricos, los resultados son muy similares a cuando los hemos escalado (entre 0 y 1), incluso ligeramente superiores en validación para los dos modelos. 
- Por tanto, de momento, este ha sido nuestro mejor modelo: BaggingRegressor con test_r2_score = 0.856.


Podemos concluir que el modelo 3.1, que utilizó la selección de variables más importantes y también incluyó el tratamiento de outliers, pero no el escalado de variables numéricas, tuvo el mejor rendimiento en términos de R2, tanto en training como en test. En este caso, la selección de variables importantes y el tratamiento de outliers han sido factores importantes para mejorar la capacidad predictiva de los modelos de BaggingRegressor y RandomForestRegressor para predecir el precio de las viviendas en Barcelona.

